# Pivoting strategies for Gaussian Elimination

### Submission by MDS202305

In [1]:
import numpy as np
import pandas as pd
import math
from Pivoting_helper_functions import *
from scipy.linalg import lu_factor,lu_solve,cho_factor,cho_solve,norm,hilbert
from IPython.display import display, HTML

The given matrix in question is the Hilbert Matrix:

<center>

$$
H_{ij} = \frac{1}{i+j-1}
$$

</center>


In [2]:
# Size of Matrix
n = [3,4,5,6,7,8,9,10,11,12,13]

# Generating the Matrix 
A = [hilbert(i) for i in n]
b = [np.random.rand(i,1) for i in n]

### Residue ($\|Ax - b\|_2$) Calculation for all solvers 

In [ ]:
# Choice of epsilon
epsilon = 1e-10

xpp, xrp, xcp, xchp = [], [], [], []
xscilu, xscich = [], []
for i in range(len(n)):
    try:
        y = gaussian_elimination_partial_pivot(A[i].copy(), b[i].copy(), epsilon)
        if y is None or not np.all(np.isfinite(y)):
            xpp.append("NaN")
        else:
            xpp.append(np.linalg.norm(A[i].copy() @ y - b[i].copy()))
    except ValueError as e:
        xpp.append("NaN")
    xrp.append("NaN") if (y := gaussian_elimination_rook_pivot(A[i].copy(), b[i].copy(), epsilon)) is None or not np.all(np.isfinite(y)) else xrp.append(np.linalg.norm(A[i].copy()@y - b[i].copy()))
    xcp.append("NaN") if (y := gaussian_elimination_complete_pivot(A[i].copy(), b[i].copy(), epsilon)) is None or not np.all(np.isfinite(y)) else xcp.append(np.linalg.norm(A[i].copy()@y - b[i].copy()))
    xchp.append(np.linalg.norm(A[i].copy()@gaussian_elimination_cholesky_pivot(A[i].copy(),b[i].copy(),epsilon)-b[i].copy()))
    lu, piv = lu_factor(A[i].copy())
    xscilu.append(np.linalg.norm(A[i].copy()@lu_solve((lu, piv), b[i].copy()) - b[i].copy()))
    try:
        q, low = cho_factor(A[i].copy())
        xscich.append(np.linalg.norm(A[i].copy() @ cho_solve((q, low), b[i].copy())  - b[i].copy()))
    except np.linalg.LinAlgError:
        xscich.append(np.nan)
        
xpp2, xrp2, xcp2, xchp2 = [], [], [], []
xscilu2, xscich2 = [], []
for i in range(len(n)):
    try:
        y = gaussian_elimination_partial_pivot(A[i].copy(), b[i].copy(), 0)
        if y is None or not np.all(np.isfinite(y)):
            xpp2.append("NaN")
        else:
            xpp2.append(np.linalg.norm(A[i].copy() @ y - b[i].copy()))
    except ValueError as e:
        xpp2.append("NaN")
    xrp2.append("NaN") if (y := gaussian_elimination_rook_pivot(A[i].copy(), b[i].copy(), 0)) is None or not np.all(np.isfinite(y)) else xrp2.append(np.linalg.norm(A[i].copy()@y - b[i].copy()))
    xcp2.append("NaN") if (y := gaussian_elimination_complete_pivot(A[i].copy(), b[i].copy(), 0)) is None or not np.all(np.isfinite(y)) else xcp2.append(np.linalg.norm(A[i].copy()@y - b[i].copy()))
    xchp2.append(np.linalg.norm(A[i].copy()@gaussian_elimination_cholesky_pivot(A[i].copy(),b[i].copy(), 0)-b[i].copy()))
    lu, piv = lu_factor(A[i].copy())
    xscilu2.append(np.linalg.norm(A[i].copy()@lu_solve((lu, piv), b[i].copy()) - b[i].copy()))
    try:
        q, low = cho_factor(A[i].copy())
        xscich2.append(np.linalg.norm(A[i].copy() @ cho_solve((q, low), b[i].copy())  - b[i].copy()))
    except np.linalg.LinAlgError:
        xscich2.append(np.nan)

In [4]:
columns = ["Matrix Order", "Norm for Partial Pivoting", "Norm for Rook Pivoting", "Norm for Complete Pivoting",
           "Norm for Cholesky Method", "SciPy LU Solver", "SciPy Cholesky Solver"]
table = (pd.DataFrame(list(zip(n, xpp, xrp, xcp, xchp, xscilu, xscich)), columns=columns)
            .set_index("Matrix Order")
            .style.set_properties(**{'text-align': 'center'})
            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])]))

table2 = (pd.DataFrame(list(zip(n, xpp2, xrp2, xcp2, xchp2, xscilu2, xscich2)), columns=columns)
             .set_index("Matrix Order")
            .style.set_properties(**{'text-align': 'center'})
            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])]))

table.set_caption("<h2>Comparison of Linear System Solvers with Tolerance &epsilon; = 1e-10</h2>")
table2.set_caption("<h2>Comparison of Linear System Solvers with Tolerance &epsilon; = 0</h2>")

table_html = table.to_html()
table2_html = table2.to_html()

html_combined = f'<div>{table2_html}</div><br><div>{table_html}</div>'
display(HTML(html_combined))

,Norm for Partial Pivoting,Norm for Rook Pivoting,Norm for Complete Pivoting,Norm for Cholesky Method,SciPy LU Solver,SciPy Cholesky Solver
Matrix Order,,,,,,
3,0.402013,0.402013,0.402013,0.402013,0.000000,0.000000
4,1.963154,1.963154,1.963154,1.963154,0.000000,0.000000
5,2.659036,2.659036,2.659036,2.659036,0.000000,0.000000
6,0.913419,0.913419,0.913419,0.913419,0.000000,0.000000
7,3.359032,3.359032,3.359032,3.359032,0.000000,0.000000
8,2.093952,2.093952,2.093952,2.093952,0.000000,0.000000
9,2.721133,2.721133,2.721134,2.721131,0.000003,0.000002
10,3.817467,3.817467,3.817396,3.817361,0.000098,0.000085
11,3.115616,3.115616,3.113845,3.113572,0.003676,0.001674


<center>
NaN or nan is returned due to the choice of &epsilon; which stops us from getting piviot further.
<center>

## Bound on Condition Number (With &epsilon; = 0)

We calculate the bound using
<center>

$\frac{\|x\|}{\|\Delta x\|} \leq \text{cond}(A) \frac{\|b\|}{\|\Delta b\|} \implies \text{cond}(A) \geq \frac{\|x\| \cdot \|\Delta b\|}{\|\Delta x\| \cdot \|b\|}$


<Center>

In [5]:
PPCN, RPCN, CPCN, CHPCN = [], [], [], []
for i in range(len(n)):
    x = gaussian_elimination_partial_pivot(A[i].copy(),b[i].copy(),0)
    δb = np.random.rand(n[i],1)*(1e-3)
    δx = gaussian_elimination_partial_pivot(A[i].copy(),b[i].copy()+δb,0)
    PPCN.append(np.linalg.norm(δx-x,ord=2)*np.linalg.norm(b[i].copy(),ord=2)/(np.linalg.norm(δb,ord=2)*(np.linalg.norm(x,ord=2))))
    x = gaussian_elimination_rook_pivot(A[i].copy(),b[i].copy(),0)
    δb = np.random.rand(n[i],1)*(1e-3)
    δx = gaussian_elimination_rook_pivot(A[i].copy(),b[i].copy()+δb,0)
    RPCN.append(np.linalg.norm(δx-x,ord=2)*np.linalg.norm(b[i].copy(),ord=2)/(np.linalg.norm(δb,ord=2)*(np.linalg.norm(x,ord=2))))
    x = gaussian_elimination_complete_pivot(A[i].copy(),b[i].copy(),0)
    δb = np.random.rand(n[i],1)*(1e-3)
    δx = gaussian_elimination_complete_pivot(A[i].copy(),b[i].copy()+δb,0)
    CPCN.append(np.linalg.norm(δx-x,ord=2)*np.linalg.norm(b[i].copy(),ord=2)/(np.linalg.norm(δb,ord=2)*(np.linalg.norm(x,ord=2))))
    x = gaussian_elimination_cholesky_pivot(A[i].copy(),b[i].copy(),0)
    δb = np.random.rand(n[i],1)*(1e-3)
    δx = gaussian_elimination_cholesky_pivot(A[i].copy(),b[i].copy()+δb,0)
    CHPCN.append(np.linalg.norm(δx-x,ord=2)*np.linalg.norm(b[i].copy(),ord=2)/(np.linalg.norm(δb,ord=2)*(np.linalg.norm(x,ord=2))))
    

e:\LAA Assignments\Assignment 4\Pivoting_helper_functions.py:144: RuntimeWarning: invalid value encountered in sqrt
  L[i, i] = np.sqrt(A[i, i] - np.sum(L[i, :i]**2))


In [6]:
columns = ["Matrix Order", "On Partial Pivoting", "On Rook Pivoting", "On Complete Pivoting", "On Cholesky Method"]

table = (pd.DataFrame(list(zip(n, PPCN, RPCN, CPCN, CHPCN)), columns=columns)
            .set_index("Matrix Order")
            .style.set_properties(**{'text-align': 'center'})
            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])]))

table.set_caption("<h2>Lower Bound on Condition Number</h2>")
table

,On Partial Pivoting,On Rook Pivoting,On Complete Pivoting,On Cholesky Method
Matrix Order,,,,
3,7.848564,4.716750,2.411432,3.386331
4,1.474944,0.458658,0.591010,0.592508
5,11.603125,9.521659,8.044772,1.788755
6,0.239364,4.924729,4.077432,5.944041
7,1.378265,0.369598,1.222187,0.698922
8,1.067958,2.600175,0.792668,1.353818
9,2.021435,0.168206,0.673144,0.584596
10,1.183564,0.064168,2.819431,0.514153
11,1.917098,1.228722,1.049246,0.536653


In [7]:
print("Using this data, the maximum lower bound on condition number for:")
print(f"The Partial Pivoting Method is {math.ceil(max(PPCN))}")
print(f"The Partial Rook Method is {math.ceil(max(RPCN))}")
print(f"The Partial Complete Method is {math.ceil(max(CPCN))}")
print(f"The Cholesky Method is {math.ceil(max(CHPCN))}")


Using this data, the maximum lower bound on condition number for:
The Partial Pivoting Method is 12
The Partial Rook Method is 10
The Partial Complete Method is 12
The Cholesky Method is 13


## Remark:
If we are to discuss the condition number for general $n \times n$ Hilbert matrices, it grows as 

<center>

$
O\left(\left(1+\sqrt{2}\right)^{4n}/\sqrt{n}\right)
$

<center>

